# Comparison of two CNN
- Tadhg Ryan 21310408
- <>

##### Code executes to end with no errors

## Resources:
- 

In [ ]:
# Import dataset
import kagglehub

# Download latest version
dataset_path = kagglehub.dataset_download("muratkokludataset/rice-image-dataset") + "\\Rice_Image_Dataset"

print("Path to dataset files:", dataset_path)


In [ ]:
# Hyperparameters
BATCH_SIZE = 64
IMG_HEIGHT = 250
IMG_WIDTH = 250
K = 5
EPOCHS = 10
LEARNING_RATE = 0.001
MAX_SIZE_DATASET = 75000


In [ ]:
# Load data in
import tensorflow as tf

# Create the full dataset (without splitting for validation)
full_dataset = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    color_mode='grayscale',  # Load labels in grayscale
    shuffle=True
)

# Get class names for later use
class_names = full_dataset.class_names

full_datasset = full_dataset.take(MAX_SIZE_DATASET // BATCH_SIZE)


In [ ]:
# Visualisation
import matplotlib.pyplot as plt

num_images = 9

# Select a sample of images to display
images_list = []
labels_list = []

# Iterate through the dataset and collect enough images
i = 0
for image_batch, label_batch in full_dataset:
    if i >= num_images:
        break
    images_list.append(image_batch)
    labels_list.append(label_batch)
    i = i + 1
if i != num_images:
    num_images = i

# Concatenate the batches into a single array
images_array = tf.concat(images_list, axis=0)
labels_array = tf.concat(labels_list, axis=0)

# Now, select the first `num_images` images and labels to display
images_to_display = images_array[:num_images]
labels_to_display = labels_array[:num_images]

plt.figure(figsize=(10, 10))

# Plot the images in a grid
for i in range(num_images):
	ax = plt.subplot(3, 3, i + 1)  # 3 rows, 3 columns
	plt.imshow(images_to_display[i].numpy().squeeze(), cmap='gray')  # Convert tensor to numpy array and display
	plt.title(class_names[labels_to_display[i].numpy()])  # Use class names for titles
	plt.axis("off")  # Hide axis

plt.tight_layout()
plt.show()


In [ ]:
# # Data Augmentation
# def preprocess_image(image, label):
#     # Explicitly reshape to correct size
#     image = tf.image.resize(image, [IMG_HEIGHT, IMG_WIDTH])
#     image = tf.reshape(image, [IMG_HEIGHT, IMG_WIDTH, 1])  # For grayscale images
#     return image, label

# # Apply the preprocessing function
# full_dataset = full_dataset.map(preprocess_image)


In [ ]:
# Create Model 1

# Step 1: Import necessary libraries
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dense, Dropout, Concatenate, Input
from tensorflow.keras.models import Model

# Step 2: Define the Inception module
def inception_module(x, filters_1x1, filters_3x3_reduce, filters_3x3, filters_5x5_reduce, filters_5x5, filters_pool_proj):
    # 1x1 Convolution branch
    conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu')(x)

    # 3x3 Convolution branch
    conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu')(x)
    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu')(conv_3x3)

    # 5x5 Convolution branch
    conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu')(x)
    conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu')(conv_5x5)

    # 3x3 MaxPooling branch
    pool_proj = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu')(pool_proj)

    # Concatenate all branches
    output = Concatenate(axis=-1)([conv_1x1, conv_3x3, conv_5x5, pool_proj])
    return output

# Step 3: Define the GoogleLeNet model
def googlenet(input_shape=(250, 250, 1), num_classes=5):
    # Input layer
    input_layer = Input(shape=input_shape)

    # Initial layers (similar to VGG)
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', activation='relu')(input_layer)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = Conv2D(64, (1, 1), padding='same', activation='relu')(x)
    x = Conv2D(192, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    # Inception modules
    x = inception_module(x, 64, 96, 128, 16, 32, 32)
    x = inception_module(x, 128, 128, 192, 32, 96, 64)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    x = inception_module(x, 192, 96, 208, 16, 48, 64)
    x = inception_module(x, 160, 112, 224, 24, 64, 64)
    x = inception_module(x, 128, 128, 256, 24, 64, 64)
    x = inception_module(x, 112, 144, 288, 32, 64, 64)
    x = inception_module(x, 256, 160, 320, 32, 128, 128)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    x = inception_module(x, 256, 160, 320, 32, 128, 128)
    x = inception_module(x, 384, 192, 384, 48, 128, 128)

    # Average pooling layer
    x = AveragePooling2D((7, 7), strides=(1, 1), padding='valid')(x)

    # Dropout layer
    x = Dropout(0.4)(x)

    # Fully connected layer
    x = Flatten()(x)
    x = Dense(num_classes, activation='softmax', name="outputs")(x)

    # Model
    model = Model(input_layer, x, name="GoogleLeNet")
    return model

# Step 4: Instantiate the model
googleLeNetModel = googlenet(input_shape=(IMG_HEIGHT, IMG_WIDTH, 1), num_classes=len(class_names))

# Display model summary
googleLeNetModel.summary()


In [ ]:
# Create Model 2
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.models import Sequential

def SimpleNet(input_shape=(250, 250, 1), num_classes=5):
    model = Sequential([
    	# Input layer
    	Input(shape=input_shape),
		Conv2D(filters=16, kernel_size=(7, 7), activation="relu"),
		MaxPooling2D((2,2)),
		Conv2D(filters=32, kernel_size=(5, 5), activation="relu"),
		MaxPooling2D((2,2)),
		Conv2D(filters=64, kernel_size=(3, 3), activation="relu"),
		MaxPooling2D((2,2)),
		Conv2D(filters=128, kernel_size=(3, 3),activation="relu"),
		MaxPooling2D((2,2)),

    	Flatten(),
     
    	# Dropout layer
    	Dropout(0.4),
     
    	# Fully connected layer
    	Dense(num_classes, activation='softmax', name="outputs"),
	])

    return model

SimpleNet().summary()


In [ ]:
# Defining k-fold cross validation
from sklearn.model_selection import KFold
import numpy as np

def RunKFold(modelFunction, optimiser, loss_function):
	AUTOTUNE = tf.data.AUTOTUNE
	full_dataset.cache().prefetch(tf.data.AUTOTUNE)

	# Create a KFold object
	kf = KFold(n_splits=K, shuffle=True)
 
	dataset_size = len(full_dataset)
	indices = np.arange(dataset_size)
	
	# Prepare to collect results
	fold_results = []

	# Iterate over K folds
	for fold, (train_index, val_index) in enumerate(kf.split(indices)):
		print(f"==================== Fold: {fold+1} ====================")
		train = full_dataset.take(1)
		for i in range(len(train_index)):
			train = full_dataset.skip(train_index[i-1]).take(1).concatenate(train)
		train.cache().prefetch(tf.data.AUTOTUNE)
   
		val = full_dataset.take(1)
		for i in range(len(val_index)):
			val = full_dataset.skip(val_index[i-1]).take(1).concatenate(val)
		val.cache().prefetch(tf.data.AUTOTUNE)

  		# Recreate the model to avoid reusing weights
		model = modelFunction(input_shape=(IMG_HEIGHT, IMG_WIDTH, 1), num_classes=len(class_names))
	
		# Compile the model
		model.compile(optimizer=optimiser, loss=loss_function, metrics=['accuracy'])

		# Train the model on the training dataset
		model.fit(train, epochs=EPOCHS, validation_data=val)

		# Evaluate on the validation dataset and store the results
		val_loss, val_accuracy = model.evaluate(val)
		fold_results.append(val_accuracy)

	# Output the results
	print(f'Cross-Validation Results: {fold_results}')
	print(f'Mean Accuracy: {sum(fold_results) / K}')
	return fold_results


In [ ]:
# Run Models with k-fold cross validation
from tensorflow.keras import optimizers, losses

optimiser = optimizers.Adam(learning_rate=LEARNING_RATE)
loss_function = losses.SparseCategoricalCrossentropy()

# RunKFold(googlenet, optimiser, loss_function)
RunKFold(SimpleNet, optimiser, loss_function)


In [ ]:
# Calculate and display metrics

In [ ]:
# Compare metrics